In [1]:
!pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 3.0/3.0 MB 58.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 43.3 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.18.1-py3-none-any.whl size=139097 sha256=ef7d9648a85a8ce35fbb1e63122f99dd6a856c6be26c83e75514c6ca4b1c620a
  Stored in directory: c:\users\diego\appdata\local\pip\cache\wheels\1a\57\6a\bb71346381d0d911cd4ce3026f1fa720da76707e4f01cf27dd
Successfully built peewee


In [3]:
import yfinance as yf
import pandas as pd

In [4]:
# Deslimitar display Pandas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [5]:
#Creamos DataFrame vacío
df_USD_X = pd.DataFrame()

# Creamos el rango de fechas para los años 2202 y 2023
# Tomamos algo de margen para tener datos al inicio de 2022 y para salidas en 2024 que entraron a final de 2023
primera_fecha = '2018-01-01'
ultima_fecha = '2024-02-01' # Comprobado y no es fin de semana, no empezaremos con nan
df_USD_X['FECHA'] = pd.date_range(start = primera_fecha, end = ultima_fecha, freq='D')

In [6]:
# Monedas de las que queremos obtener el tipo
# ['Dolar', '', 'Real', 'Dolarjamaicano', 'PesoDominicano']
lista_monedas = [
    'JMD', # Dolar Jamaicano
    'DOP', # Peso Dominicano
    'MXN', # Peso Mexicano
    'BRL', # Real Brasileño
]

# Para el dolar ponemos un 1 en toda la columna
df_USD_X['USD'] = 1

lista_monedas

['JMD', 'DOP', 'MXN', 'BRL']

In [7]:
# Creación de las columnas para cada moneda

for moneda in lista_monedas:

    # Descargar datos
    df_moneda = yf.download(f"{moneda}=X", start = primera_fecha, end = ultima_fecha, interval="1d")
    df_moneda = df_moneda[['Close']]

    # Reindexar para tener todos los días del año
    df_moneda = df_moneda.reindex(df_USD_X['FECHA'])

    # Rellenar valores faltantes con el último valor disponible hacia adelante (forward fill) para fines de semana y festivos
    df_moneda['Close'] = df_moneda['Close'].ffill()

    # Quitar multiindex
    df_moneda = pd.DataFrame(df_moneda.values)

    # Nombro la columna
    df_moneda.columns = [f"{moneda}"]

    # Concatenación
    df_USD_X = pd.concat([df_USD_X, df_moneda], axis=1)

C:\Users\Diego\AppData\Local\Temp\ipykernel_15604\364419150.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_moneda = yf.download(f"{moneda}=X", start = primera_fecha, end = ultima_fecha, interval="1d")
[*********************100%***********************]  1 of 1 completed
C:\Users\Diego\AppData\Local\Temp\ipykernel_15604\364419150.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_moneda = yf.download(f"{moneda}=X", start = primera_fecha, end = ultima_fecha, interval="1d")
[*********************100%***********************]  1 of 1 completed
C:\Users\Diego\AppData\Local\Temp\ipykernel_15604\364419150.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_moneda = yf.download(f"{moneda}=X", start = primera_fecha, end = ultima_fecha, interval="1d")
[*********************100%***********************]  1 of 1 completed
C:\Users\Diego\AppData\Local\Temp\ipykernel_15604\364419150.py:6

In [8]:
# Ya tenemos el DF de conversión

print("Un USD equivale a:")
df_USD_X.head(10)

Un USD equivale a:


,FECHA,USD,JMD,DOP,MXN,BRL
0,2018-01-01,1,122.900002,47.618999,19.650850,3.3076
1,2018-01-02,1,122.779999,47.743000,19.497400,3.3110
2,2018-01-03,1,122.849998,47.734001,19.351101,3.2578
3,2018-01-04,1,122.540001,47.733002,19.306971,3.2355
4,2018-01-05,1,122.379997,47.886002,19.174000,3.2331
5,2018-01-06,1,122.379997,47.886002,19.174000,3.2331
6,2018-01-07,1,122.379997,47.886002,19.174000,3.2331
7,2018-01-08,1,122.739998,47.866001,19.192120,3.2266
8,2018-01-09,1,122.680000,47.755001,19.244680,3.2390
9,2018-01-10,1,122.220001,48.019001,19.263000,3.2468


In [9]:
df_USD_X.to_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\USD_X.csv', index=False)